# ⚡ Lab 2.3: Model Architecture Comparison
**Module 3: Computer Vision and Image Processing**
B-Tech AI Specialization | Chitkara University | February 2026

---

## 🍓 Industry Scenario
> A client wants to deploy a classifier on a **Raspberry Pi** — limited CPU, no GPU. You need to recommend which model architecture to use. The choice requires balancing accuracy with inference speed and memory. You need **real benchmark data** to make the recommendation.

## 🎯 Objective
Benchmark **VGG16**, **ResNet50**, and **MobileNetV2** on inference speed and model size. Recommend which to use for edge deployment.

**Time:** 60 minutes | **Mode:** Individual

---

## ⚙️ Setup — Run First

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import ipywidgets as widgets
from IPython.display import display, HTML, Code

import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2

print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")
print("✅ Ready")

In [ ]:
def reveal_button(hint_text, solution_code):
    import ipywidgets as widgets
    from IPython.display import display, HTML, Code
    out = widgets.Output()
    hint_btn = widgets.Button(description='💡 Hint', button_style='info',
        layout=widgets.Layout(width='120px', margin='4px'))
    sol_btn  = widgets.Button(description='✅ Solution', button_style='warning',
        layout=widgets.Layout(width='140px', margin='4px'))
    hide_btn = widgets.Button(description='🙈 Hide', button_style='',
        layout=widgets.Layout(width='100px', margin='4px'))
    def on_hint(b):
        with out:
            out.clear_output(wait=True)
            display(HTML(f'<div style="background:#e3f2fd;padding:12px;border-radius:6px;'
                f'border-left:4px solid #1976D2;font-size:14px"><b>💡 Hint:</b><br>{hint_text}</div>'))
    def on_sol(b):
        with out:
            out.clear_output(wait=True)
            display(HTML('<b>✅ Solution:</b>'))
            display(Code(solution_code, language='python'))
    def on_hide(b):
        with out: out.clear_output()
    hint_btn.on_click(on_hint); sol_btn.on_click(on_sol); hide_btn.on_click(on_hide)
    display(widgets.HBox([hint_btn, sol_btn, hide_btn]), out)

print("reveal_button() ready ✅")

---
## 🤔 Predict Before You Benchmark

Before writing any code, fill in your guesses in the table below. Commit to a number — we'll compare against real results.

| Model | Your guess: # Parameters | Your guess: Inference time (ms) | Best for? |
|---|---|---|---|
| VGG16 | ? M | ? ms | ? |
| ResNet50 | ? M | ? ms | ? |
| MobileNetV2 | ? M | ? ms | ? |

Also answer:
1. What is a "residual connection" (ResNet's key innovation)?
2. What makes MobileNet "mobile" — what did the designers sacrifice?
3. On a Raspberry Pi with no GPU, does model size or architecture matter more?

In [ ]:
# ✏️ Your predictions:
# VGG16:       params ~_M,  latency ~_ms
# ResNet50:    params ~_M,  latency ~_ms
# MobileNetV2: params ~_M,  latency ~_ms

# 1. Residual connections are...
# 2. MobileNet is "mobile" because...
# 3. On CPU, what matters most is...

---
## Task 2: Load All 3 Models

Load each model with ImageNet weights. We use `include_top=True` so we get the full model including the classifier head — this gives us realistic parameter counts.

> ⚠️ This will download ~700 MB total. It takes 2–3 minutes on Colab — that's expected.

In [ ]:
# TODO: Load all 3 models with imagenet weights
# VGG16 expects 224x224, ResNet50 expects 224x224, MobileNetV2 expects 224x224

vgg16       = None  # VGG16(weights=..., include_top=...)
resnet50    = None  # ResNet50(...)
mobilenetv2 = None  # MobileNetV2(...)

models_dict = {
    'VGG16':       vgg16,
    'ResNet50':    resnet50,
    'MobileNetV2': mobilenetv2,
}

# Quick sanity check — print total params for each
for name, model in models_dict.items():
    if model is not None:
        print(f"{name:<15}: {model.count_params():>12,} parameters")

In [ ]:
reveal_button(
    hint_text="All three use the same pattern: <code>ModelName(weights='imagenet', include_top=True)</code>. "
              "MobileNetV2 input defaults to 224x224 so no need to specify input_shape.",
    solution_code=(
        "vgg16       = VGG16(weights='imagenet', include_top=True)\n"
        "resnet50    = ResNet50(weights='imagenet', include_top=True)\n"
        "mobilenetv2 = MobileNetV2(weights='imagenet', include_top=True)\n\n"
        "models_dict = {'VGG16': vgg16, 'ResNet50': resnet50, 'MobileNetV2': mobilenetv2}"
    )
)

---
## Task 3: Measure Inference Latency

Run **100 inference passes** on a dummy image for each model. Average them to get stable latency numbers.

> 💡 **Why 100 passes?** The first few calls are slower (GPU/memory warmup). Averaging over 100 gives a stable, representative number.

In [ ]:
# Create a dummy input image — same shape that all 3 models expect
dummy_input = np.random.rand(1, 224, 224, 3).astype(np.float32)

N_RUNS = 100
results = {}

for name, model in models_dict.items():
    if model is None:
        print(f"⚠️  {name} not loaded — skipping")
        continue

    # TODO: Warm up the model (run once before timing)
    # model.predict(dummy_input, verbose=0)

    # TODO: Time N_RUNS inference passes
    # start = time.time()
    # for _ in range(N_RUNS):
    #     model.predict(dummy_input, verbose=0)
    # elapsed = time.time() - start

    elapsed = 0  # replace with real timing

    avg_ms  = (elapsed / N_RUNS) * 1000
    fps     = 1000 / avg_ms if avg_ms > 0 else 0
    params  = model.count_params()

    results[name] = {
        'Parameters (M)':  round(params / 1e6, 1),
        'Latency (ms)':    round(avg_ms, 1),
        'FPS':             round(fps, 1),
    }
    print(f"{name}: {avg_ms:.1f} ms/inference  ({fps:.1f} FPS)  |  {params/1e6:.1f}M params")

In [ ]:
reveal_button(
    hint_text="Call <code>model.predict(dummy_input, verbose=0)</code> once before the loop (warmup). "
              "Then use <code>time.time()</code> before and after the loop to measure total elapsed time. "
              "Average: <code>elapsed / N_RUNS * 1000</code> gives ms per inference.",
    solution_code=(
        "for name, model in models_dict.items():\n"
        "    model.predict(dummy_input, verbose=0)  # warmup\n"
        "    start = time.time()\n"
        "    for _ in range(N_RUNS):\n"
        "        model.predict(dummy_input, verbose=0)\n"
        "    elapsed = time.time() - start\n"
        "    avg_ms = (elapsed / N_RUNS) * 1000\n"
        "    fps    = 1000 / avg_ms\n"
        "    params = model.count_params()\n"
        "    results[name] = {\n"
        "        'Parameters (M)': round(params/1e6, 1),\n"
        "        'Latency (ms)':   round(avg_ms, 1),\n"
        "        'FPS':            round(fps, 1),\n"
        "    }"
    )
)

---
## Task 4: Build & Display the Comparison Table

In [ ]:
# TODO: Create a pandas DataFrame from the results dict and display it

# df = pd.DataFrame(results).T
# df.index.name = 'Model'

# Style the table to highlight the best value in each column
# df.style.highlight_min(axis=0, color='lightgreen')  # lower = better for params & latency

df = None  # replace with real DataFrame

if df is not None:
    display(df)

In [ ]:
reveal_button(
    hint_text="<code>pd.DataFrame(results).T</code> transposes the dict into rows=models, cols=metrics. "
              "Then <code>.style.highlight_min(axis=0, color='lightgreen')</code> highlights the best per column.",
    solution_code=(
        "df = pd.DataFrame(results).T\n"
        "df.index.name = 'Model'\n"
        "display(df.style.highlight_min(axis=0, color='lightgreen')\n"
        "           .highlight_max(axis=0, color='#ffcccc')\n"
        "           .format(precision=1))"
    )
)

---
## 🎚️ Task 5: Interactive Benchmark Explorer

Use the controls to explore different views of the benchmark data. Which model is best depends on what you optimise for — use this to build your intuition.

In [ ]:
x_axis = widgets.Dropdown(
    options=[('Parameters (M)', 'Parameters (M)'), ('Latency (ms)', 'Latency (ms)'), ('FPS', 'FPS')],
    value='Parameters (M)', description='X axis:', layout=widgets.Layout(width='250px')
)
y_axis = widgets.Dropdown(
    options=[('Latency (ms)', 'Latency (ms)'), ('FPS', 'FPS'), ('Parameters (M)', 'Parameters (M)')],
    value='Latency (ms)', description='Y axis:', layout=widgets.Layout(width='250px')
)
chart_type = widgets.ToggleButtons(
    options=['Scatter', 'Bar'], description='Chart:', button_style='info'
)
out_chart = widgets.Output()

COLORS = {'VGG16': '#e74c3c', 'ResNet50': '#3498db', 'MobileNetV2': '#2ecc71'}

def update_chart(change=None):
    with out_chart:
        out_chart.clear_output(wait=True)
        if not results:
            print("⚠️  Run Tasks 2 and 3 first to populate results.")
            return

        fig, ax = plt.subplots(figsize=(9, 5))

        if chart_type.value == 'Scatter':
            for model_name, vals in results.items():
                ax.scatter(vals[x_axis.value], vals[y_axis.value],
                           s=200, color=COLORS[model_name], label=model_name, zorder=5)
                ax.annotate(f"  {model_name}",
                            (vals[x_axis.value], vals[y_axis.value]),
                            fontsize=11, fontweight='bold', color=COLORS[model_name])
            ax.set_xlabel(x_axis.value, fontsize=12)
            ax.set_ylabel(y_axis.value, fontsize=12)
        else:
            model_names = list(results.keys())
            vals = [results[m][y_axis.value] for m in model_names]
            bars = ax.bar(model_names, vals, color=[COLORS[m] for m in model_names], width=0.5)
            for bar, val in zip(bars, vals):
                ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(vals)*0.01,
                        f'{val}', ha='center', fontsize=11, fontweight='bold')
            ax.set_ylabel(y_axis.value, fontsize=12)

        ax.set_title(f'Model Comparison: {y_axis.value}', fontsize=13, fontweight='bold')
        ax.grid(True, alpha=0.3, axis='y')
        if chart_type.value == 'Scatter': ax.legend(fontsize=11)
        plt.tight_layout()
        plt.show()

x_axis.observe(update_chart, names='value')
y_axis.observe(update_chart, names='value')
chart_type.observe(update_chart, names='value')

display(widgets.VBox([
    widgets.HBox([x_axis, y_axis, chart_type]),
    out_chart
]))
update_chart()

---
## ✍️ Task 6: Your Recommendation

Based on the benchmark data, write **3 sentences** recommending which model to deploy on the Raspberry Pi. Your answer should justify:
- Why you chose that model (cite specific numbers)
- What you're sacrificing (every choice has a trade-off)
- One condition under which you'd choose a different model instead

In [ ]:
# ✏️ Your recommendation:
# I recommend ___ because...
# The trade-off is...
# I would switch to ___ if...

---
## 🤔 Compare Against Your Predictions

Go back to your predictions at the top. How close were you?

| Model | Predicted params | Actual params | Predicted latency | Actual latency |
|---|---|---|---|---|
| VGG16 | ? | | ? | |
| ResNet50 | ? | | ? | |
| MobileNetV2 | ? | | ? | |

**Which result surprised you the most? Why?**

In [ ]:
# ✏️ What surprised me most:
# I was most wrong about...
# This makes sense because...